In [1]:
import pandas as pd

In [2]:
distfi = pd.read_csv('distribucion_fuente_ingreso.csv', sep=';', decimal=',')
indgini = pd.read_csv('indice_gini.csv', sep=';', decimal=',')
inddemo = pd.read_csv('indices_demograficos.csv', sep=';', decimal=',')
rentamedia = pd.read_csv('renta_media_y_mediana.csv', sep=';', decimal='.')
pisos_turisticos = pd.read_csv('exp_viv_turistica_tabla5_FEB2021.csv', sep=';', decimal=',')

In [3]:
new_data = distfi["location_raw"].str.split(" ", n = 4, expand = True)
distfi["Code"] = new_data[0].astype(int)

In [4]:
new_data = indgini["location_raw"].str.split(" ", n = 4, expand = True)
indgini["Code"] = new_data[0].astype(int)

In [5]:
new_data = inddemo["location_raw"].str.split(" ", n = 4, expand = True)
inddemo["Code"] = new_data[0].astype(int)

In [6]:
new_data = rentamedia["location_raw"].str.split(" ", n = 4, expand = True)
rentamedia["Code"] = new_data[0].astype(int)

In [7]:
distfi_filtered = distfi[distfi['Code'].between(4625000000, 4625999999)]
indgini_filtered = indgini[indgini['Code'].between(4625000000, 4625999999)]
inddemo_filtered = inddemo[inddemo['Code'].between(4625000000, 4625999999)]
rentamedia_filtered = rentamedia[rentamedia['Code'].between(4625000000, 4625999999)]
pisos_turisticos_filtered = pisos_turisticos[pisos_turisticos['CODIGO'].between(4625000000, 4625999999)]

In [8]:
distfi_filtered.drop('location_raw', axis=1, inplace=True)
indgini_filtered.drop('location_raw', axis=1, inplace=True)
inddemo_filtered.drop('location_raw', axis=1, inplace=True)
rentamedia_filtered.drop('location_raw', axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
full_dtst = pd.merge(distfi_filtered, indgini_filtered, on="Code")

In [10]:
full_dtst = pd.merge(full_dtst, inddemo_filtered, on="Code")

In [11]:
full_dtst = pd.merge(full_dtst, rentamedia_filtered, on="Code")

In [12]:
pisos_turisticos_filtered.rename(columns = {'CODIGO':'Code'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
full_ine_dtst = pd.merge(full_dtst, pisos_turisticos_filtered, on="Code")

In [14]:
full_ine_dtst.shape

(607, 28)

In [15]:
full_ine_dtst.fillna(value=0,inplace=True)

In [16]:
full_ine_dtst.rename(columns = {'Fuente de ingreso: salario':'Asalariados', 'Fuente de ingreso: pensiones':'Pensionistas', 'Fuente de ingreso: prestaciones por desempleo':'Desempleados', 'Fuente de ingreso: otras prestaciones':'Prestacionistas', 'Fuente de ingreso: otros ingresos':'Otros_ingresos', 'Índice de Gini':'Index_Gini','Distribución de la renta P80/P20':'dist_rent_p80_p20','Edad media de la población':'media_edad', 'Porcentaje de población menor de 18 años':'percent_menos_18', 'Porcentaje de población de 65 y más años':'percernt_mas_65', 'Tamaño medio del hogar':'tamaño_casa_media', 'Porcentaje de hogares unipersonales':'percent_casa_unipersonales', 'Población':'habitantes', 'Renta neta media por persona':'renta_media_persona', 'Renta neta media por hogar':'renta_media_hogar', 'Media de la renta por unidad de consumo':'media_unidad_consumo', 'Mediana de la renta por unidad de consumo':'mediana_unidad_consumo', 'Renta bruta media por persona':'renta_bruta_persona','Renta bruta media por hogar':'renta_bruta_hogar','VIVIENDA TURISTICA':'viviendas_turisticas', 'PLAZAS':'plazas', 'PLAZA POR VIVIENDA TURISTICA':'plaza_vivienda_turistica', 'PORCENTAJE VIVIENDA TURISTICA':'percent_vivienda_turistica'}, inplace = True)

In [17]:
full_ine_dtst.drop('PROV', axis=1, inplace=True)
full_ine_dtst.drop('PROV_LITERAL', axis=1, inplace=True)
full_ine_dtst.drop('MUN', axis=1, inplace=True)
full_ine_dtst.drop('MUN_LITERAL', axis=1, inplace=True)

In [18]:
full_ine_dtst['Code'] = full_ine_dtst['Code'] - 4625000000

In [19]:
full_ine_dtst['coddistrit'] = full_ine_dtst.apply(lambda row: (row['Code']/1000), axis=1) 
full_ine_dtst['coddistrit'] = full_ine_dtst['coddistrit'].astype(int)

In [20]:
full_ine_dtst['codsec'] = full_ine_dtst.apply(lambda row: (row['Code']-(row['coddistrit']*1000)), axis=1) 
full_ine_dtst['codsec'] = full_ine_dtst['codsec'].astype(int)

In [21]:
full_ine_dtst['Code'] = full_ine_dtst.apply(lambda row: (row['codsec']+(row['coddistrit']*100)), axis=1) 

In [22]:
full_ine_dtst.to_csv(r'dataset_ine_valencia.csv')